In [5]:
import pyodbc 
import pandas as pd
import numpy as np
from datetime import datetime

df = pd.read_csv("./PlayerSeason.2024.csv")

print(df.columns)
print(df.shape)

# TODO: https://www.dropbox.com/scl/fi/hy0sxw6gaai7ghemrshi8/lahman_1871-2023_csv.7z?dl=0&e=1&file_subpath=%2Flahman_1871-2023_csv&rlkey=edw1u63zzxg48gvpcmr3qpnhz

Index(['StatID', 'TeamID', 'PlayerID', 'SeasonType', 'Season', 'Name', 'Team',
       'Position', 'PositionCategory', 'Started', 'AverageDraftPosition',
       'AuctionValue', 'Games', 'FantasyPoints', 'AtBats', 'Runs', 'Hits',
       'Singles', 'Doubles', 'Triples', 'HomeRuns', 'RunsBattedIn',
       'BattingAverage', 'Outs', 'Strikeouts', 'Walks', 'HitByPitch',
       'Sacrifices', 'SacrificeFlies', 'GroundIntoDoublePlay', 'StolenBases',
       'CaughtStealing', 'OnBasePercentage', 'SluggingPercentage',
       'OnBasePlusSlugging', 'Wins', 'Losses', 'Saves',
       'InningsPitchedDecimal', 'TotalOutsPitched', 'InningsPitchedFull',
       'InningsPitchedOuts', 'EarnedRunAverage', 'PitchingHits',
       'PitchingRuns', 'PitchingEarnedRuns', 'PitchingWalks',
       'PitchingStrikeouts', 'PitchingHomeRuns', 'PitchesThrown',
       'PitchesThrownStrikes', 'WalksHitsPerInningsPitched',
       'PitchingBattingAverageAgainst', 'FantasyPointsFanDuel',
       'FantasyPointsDraftKings', 'Weight

In [ ]:
server = 'tcp:sports-iq.database.windows.net,1433' 
database = 'main' 
username = 'CloudSAeab82b26' 
password = ''

cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

In [ ]:

df = df.dropna(subset=['PositionCategory'])
df = df.replace(np.nan, None)

print(type(df['BirthDate']))

print(df.shape)

In [ ]:
for index, row in df.iterrows():
		
		cursor.execute("SELECT t.TeamID FROM [MLB].[Teams] t WHERE Abbreviation = ?", row.Team)
		team = cursor.fetchone()

		cursor.execute("SELECT t.TeamID FROM [MLB].[Players] t WHERE Abbreviation = ?", row.Team)
		team = cursor.fetchone()

		playerTeam = int(team[0]) if team is not None else None

		date = None

		if (row.BirthDate is not None):
			date = datetime.strptime(row.BirthDate, "%m/%d/%Y %H:%M:%S %p")
																																																																																												#11/30/1987 12:00:00 AM
		cursor.execute("INSERT INTO MLB.Players (FirstName,LastName,Jersey,PositionCategory,Position,Status,TeamID, DateOfBirth) VALUES(?,?,?,?,?,?,?,?)", row.FirstName, row.LastName, row.Jersey, row.PositionCategory, row.Position, row.Status, playerTeam, date)

cnxn.commit()
cursor.close()